In [ ]:
# LoRA Streamlit UI: Quickstart for Google Colab
# Kill any previous Streamlit or cloudflared processes (safe in Colab)
!pkill -f streamlit || true
!pkill -f cloudflared || true
import time
import socket
import random

# Wait a bit to ensure ports are released
time.sleep(2)

# Find a free port (prefer 8080, else pick 8081-8099)
def get_free_port(preferred=8080, fallback_range=(8081, 8099)):
    s = socket.socket()
    try:
        s.bind(("", preferred))
        port = preferred
    except OSError:
        for p in range(*fallback_range):
            try:
                s.bind(("", p))
                port = p
                break
            except OSError:
                continue
        else:
            raise RuntimeError("No free port found in range 8080-8099")
    s.close()
    return port

PORT = get_free_port()
print(f"[INFO] Using port {PORT} for Streamlit and Cloudflared.")

# 1. Clone your repo (published repo URL)
!git clone https://github.com/melbinjp/Lora_trainer.git
%cd Lora_trainer

# 2. Run the smart setup script
!bash setup.sh

# 3. Install cloudflared for public tunneling
!wget -q -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared

# 4. Start Streamlit in the background and wait for it to be ready
import os
import subprocess
import re

streamlit_proc = subprocess.Popen(
    ['streamlit', 'run', 'test.py', f'--server.port={PORT}', '--server.enableCORS=false'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT
)

# Wait for Streamlit to start (look for 'You can now view your Streamlit app')
ready = False
for i in range(60):
    line = streamlit_proc.stdout.readline().decode()
    print(line, end='')
    if 'You can now view your Streamlit app' in line:
        ready = True
        break
    time.sleep(1)
if not ready:
    print('Streamlit did not start in time. Check the logs above.')

# 5. Start cloudflared tunnel and print the public URL
if ready:
    tunnel = subprocess.Popen(['./cloudflared', 'tunnel', '--url', f'http://localhost:{PORT}'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    public_url = None
    for i in range(60):
        line = tunnel.stdout.readline().decode()
        print(line, end='')
        if 'trycloudflare.com' in line:
            public_url = re.search(r'https://[\w\-]+\.trycloudflare.com', line)
            if public_url:
                print(f'\nYour public Streamlit URL: {public_url.group(0)}')
                break
        time.sleep(1)
    if not public_url:
        print('Cloudflared tunnel failed to start. Please check the output above or try restarting the Colab runtime and running all cells again.')
else:
    print('Cloudflared tunnel not started because Streamlit did not start.')

# Note: The Streamlit UI will be available at the printed public URL above if successful.
# If the URL does not work, try restarting the Colab runtime and running all cells again.

In [ ]:
# Hardware & PyTorch Version Check (Recommended)
import torch
import platform
import os

def get_cuda_version():
    try:
        import subprocess
        output = subprocess.check_output(['nvcc', '--version']).decode()
        for line in output.split('\n'):
            if 'release' in line:
                return line.strip().split('release')[-1].split(',')[0].strip()
    except Exception:
        return None

cuda_available = torch.cuda.is_available()
mps_available = hasattr(torch.backends, 'mps') and torch.backends.mps.is_available()
torch_version = torch.__version__
cuda_version = get_cuda_version() if cuda_available else None

print(f"[INFO] Detected torch version: {torch_version}")
if cuda_available:
    print("[INFO] CUDA GPU detected.")
    print(f"[INFO] CUDA version: {cuda_version if cuda_version else 'Unknown'}")
    # Recommend best torch version for Colab CUDA 11.8
    print("[RECOMMEND] For best performance on Colab, use torch 2.0.0+cu118:")
    print("  pip install torch==2.0.0+cu118 torchvision==0.15.1+cu118 --index-url https://download.pytorch.org/whl/cu118")
elif mps_available:
    print("[INFO] Apple Silicon (MPS) detected.")
    print("[RECOMMEND] For best performance, use torch>=1.12.0:")
    print("  pip install torch>=1.12.0")
else:
    print("[INFO] No GPU/NPU detected. Using CPU.")
    print("[RECOMMEND] For best performance, use the latest torch version:")
    print("  pip install torch --upgrade")


### Important Note on UI Modes

The application now starts in **Simple Mode** by default. This mode is designed to be as user-friendly as possible, with most settings automated for you.

If you are an advanced user and need access to detailed settings like learning rate, schedulers, or custom models, you can easily switch to **Advanced Mode** using the toggle button at the top of the application's UI.